<a href="https://colab.research.google.com/github/swrp-h/NER_with_transformers/blob/main/multinerd_ner_filt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning a transformer for named entity recognition (filtered version)

This notebook is part of a project to finetune a transformer model for English NER. The dataset utilised here is the filtered version of the English subset of [Multinerd Dataset by Babelscape](https://huggingface.co/datasets/Babelscape/multinerd). Most of the NER labels have been filtered out, except for PERSON(PER), ORGANIZATION(ORG), LOCATION(LOC), DISEASES(DIS), ANIMAL(ANIM). This new dataset has been uploaded [here](https://huggingface.co/datasets/shrop/multinerd_en_filtered).

 The notebook contents have been inspired by the [official Hugging Face documentation](https://huggingface.co/docs/transformers/main/tasks/token_classification).

## Initial Setup, installations, imports




In [ ]:
# Uncomment if using Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Optional for uploading your model:
# !python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('YOUR_HF_TOKEN')"

In [ ]:
MODEL_OP_DIRECTORY = "PATH_TO_MODEL_OUTPUT_DIRECTORY"
TF_MODEL = "distilbert-base-uncased"

In [ ]:
## Installations: Do not run this cell if not on Google Colab

!pip uninstall transformers --y
!pip install transformers==4.28.0
!pip install datasets
!pip install -U accelerate
!pip install evaluate
!pip install seqeval

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0


In [ ]:
# Imports

import transformers
import accelerate
from transformers import AutoTokenizer
import evaluate
seqeval = evaluate.load("seqeval")
from datasets import load_dataset
from evaluate import evaluator
from datasets import load_dataset

In [ ]:
transformers.__version__

'4.28.0'

## Initializing and Preprocessing the data



In [ ]:
## Initializing the tokenizer
# Model used: DistilBERT for speed.

tokenizer = AutoTokenizer.from_pretrained(TF_MODEL)

def tokenize_and_align_labels(dataset):
    """
    Tokenize the input tokens and align the corresponding labels to the tokenized input.

    Args:
        dataset (Dataset): A dataset from the HF hub containing the input tokens and corresponding labels.

    Returns:
        Dataset: A tokenized version of the dataset containing the tokenized inputs with aligned labels.
    """
    tokenized_inputs = tokenizer(dataset["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(dataset[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
## tokenising the data

dataset_train = load_dataset("shrop/multinerd_en_filtered",split="train")
dataset_val = load_dataset("shrop/multinerd_en_filtered",split="validation")
dataset_test = load_dataset("shrop/multinerd_en_filtered",split="test")

tokenized_train = dataset_train.map(tokenize_and_align_labels, batched=True)
tokenized_val = dataset_val.map(tokenize_and_align_labels, batched=True)
tokenized_test = dataset_test.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/16410 [00:00<?, ? examples/s]

In [ ]:
## Setup to dynamically pad tokens and labels

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Setup for Training

In [ ]:
## Mapping ids and labels for training (labels derived from the Multinerd dataset card)

id2label = {
    "O": 0, "B-PER": 1, "I-PER": 2, "B-ORG": 3, "I-ORG": 4, "B-LOC": 5, "I-LOC": 6, "B-ANIM": 7, "I-ANIM": 8, "B-DIS": 9, "I-DIS": 10,
  }

label2id = {label: idx for idx, label in id2label.items()}

# Example usage
print(label2id)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-ANIM', 8: 'I-ANIM', 9: 'B-DIS', 10: 'I-DIS'}


In [ ]:
label_list = [k for k,v in id2label.items()]
label_list

['O',
 'B-PER',
 'I-PER',
 'B-ORG',
 'I-ORG',
 'B-LOC',
 'I-LOC',
 'B-ANIM',
 'I-ANIM',
 'B-DIS',
 'I-DIS']

In [ ]:
## Loading the model and specifying the number of labels and label mappings
## In this case, it is 11 for the total labels in the filtered Multinerd dataset for the project

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    TF_MODEL, num_labels=11, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream

In [ ]:
## Setup for evaluation after training

import numpy as np

labels = label_list

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
## Defining hyperparameters

training_args = TrainingArguments(
    output_dir=MODEL_OP_DIRECTORY,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #push_to_hub=True,             ## Uncomment if uploading model to HF
)

## Training

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/drive/MyDrive/personal_projects/rise_test/saved_models/MY_BERT_NER/my_filtered_ner_model is already a clone of https://huggingface.co/shrop/my_filtered_ner_model. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` t

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.023100,0.026149,0.932101,0.950612,0.941265,0.990816
2,0.016200,0.024405,0.941978,0.951701,0.946814,0.991761


TrainOutput(global_step=16410, training_loss=0.02639777182370406, metrics={'train_runtime': 1620.1906, 'train_samples_per_second': 162.055, 'train_steps_per_second': 10.128, 'total_flos': 3738242290705728.0, 'train_loss': 0.02639777182370406, 'epoch': 2.0})

In [ ]:
## Extra: Eval with another split

training_args = TrainingArguments(
    output_dir=MODEL_OP_DIRECTORY,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

{'eval_loss': 0.024998987093567848,
 'eval_precision': 0.9362065692044808,
 'eval_recall': 0.9426946422597142,
 'eval_f1': 0.9394394036817413,
 'eval_accuracy': 0.991775964285997,
 'eval_runtime': 42.8474,
 'eval_samples_per_second': 382.987,
 'eval_steps_per_second': 23.945}